# Sagah Tools
Esse notebook realiza checkagem de dados entre um dataset interno e um dataset da aplicação. O objetivo é verificar a paridade de dados entre os dois sistemas.

### Importação de bibliotecas

In [ ]:
import pandas as pd

### Carrega os datasets em dataframes
O arquivo excel é um relatório extraído do nosso sistema acadêmico, enquanto o arquivo CSV é um relatório extraído da Sagah.

In [ ]:
unisc = pd.read_excel('datasets/sagah_2024_1_ajustes.xlsx')
sagah_csv = pd.read_csv('datasets/turmas_online_2024-1.csv', delimiter=';')
unisc['nome_moodle'] = unisc['nome_moodle'].str.strip()
unisc['professor'] = unisc['professor'].str.strip()
unisc['email'] = unisc['email'].str.strip()

### Remove as linhas duplicadas
Quando configuradas as UAs (unidades de aprendizagem) no Catálogo, há uma duplicação de linhas contendo o nome da disciplina e o professor, únicos objetos de interesse neste trabalho. Portanto, é necessário remover as linhas duplicadas. 

In [ ]:
sagah = sagah_csv[['Curso', 'Disciplina', 'Situação', 'Coordenador(a)']].drop_duplicates()

### Verificação de disciplinas na Sagah
Como podem ocorrer lançamentos no nosso sistema acadêmico, ainda, após a exportação dos dados, é necessário realizar uma conferência para verificar se todas as disciplinas estão presentes em ambos datasets.

In [ ]:
itens_unisc_nao_sagah = unisc[~unisc['nome_moodle'].isin(sagah['Disciplina'])]['nome_moodle'].tolist()
itens_unisc_nao_sagah

### Verificação de associação de professores em disciplinas
Utilizando uma chave composta (nome da disciplina e nome do professor), verifica a paridade de informação entre os datasets.

In [ ]:
unisc_not_in_sagah = unisc[~unisc.set_index(['nome_moodle', 'professor']).index.isin(sagah.set_index(['Disciplina', 'Coordenador(a)']).index)]
unisc_dict = unisc_not_in_sagah[['nome_moodle', 'professor']].to_dict(orient='records')

unisc_dict = {item['nome_moodle']: item['professor'] for item in unisc_dict}
unisc_dict

for disciplina,professor in unisc_dict.items():
    print(f'{disciplina} - {professor}')
        

### Contabilização de diferenças
Após a verificação dos dados, realiza a contabilização das diferenças entre os datasets. Primeiro, indica quantas disciplinas aindas estão se professor. Depois, os professores indicados no sistema acadêmico mas que não estão associados na Sagah.

In [ ]:
count = sum(value == 'Sem professor' for value in unisc_dict.values())
print(f"Disciplinas sem professor: {count}")
count = sum(value != 'Sem professor' for value in unisc_dict.values())
print(f'Disciplinas com professor diferente na Sagah: {count}')

